In [2]:
import math
import os
import pandas as pd
from itertools import combinations
import scipy.cluster.hierarchy as sch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import hypergeom
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import xlsxwriter


In [3]:
# Specify the directory path where your files are located
directory_path = 'FUMA_downloads_all_tissue/eQTLs'

# Initialize an empty list to store DataFrames
df_list = []

# Initialize an empty list to store disorder names
disorder_names = []

# Iterate through files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # Process only .txt files
        # Extract disorder name from the filename
        disorder_name = filename.split('_')[1].split('.')[0]
        disorder_names.append(disorder_name)

        # Read the file as a DataFrame
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path, sep='\t')

        # Add a column with the disorder name
        df['disorder'] = disorder_name

        # Append the DataFrame to the list
        df_list.append(df)

C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')
C:\Users\annei\AppData\Local\Temp\ipykernel_10256\538097247.py:19: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
 

In [4]:
#select only the columns of interest
new_df_list = []
for df in df_list:
    df = df[['uniqID', 'tissue', 'symbol', 'disorder']]
    new_df_list.append(df)

In [6]:
#get uniqueID and rsID from SNP files
path = 'FUMA_downloads_all_tissue/SNPs'

file_list = os.listdir(path)

files_df = pd.DataFrame()

for file in file_list:
    file_path = os.path.join(path, file)

    df = pd.read_csv(file_path, sep='\t')

    files_df = pd.concat([files_df, df], ignore_index=True)

#Keep only the relevant columns
files_df = files_df[['uniqID', 'rsID']]



In [ ]:
#map the rsIDs to the SNPid in the df list of gene data
merged_list = []
for df in new_df_list:
    merged_df = pd.merge(df, files_df, on='uniqID')
    merged_list.append(merged_df)
